In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load 

file1 = "data/population-density-vs-prosperity.csv"
file2 = "data/urban-and-rural-population.csv"
file3 = "data/urban-vs-rural-majority.csv"
file4 = "data/life-expectancy-at-birth-total-years.csv"
file5 = "data/median-age.csv"
file6 = "data/six-dem_mod.csv"
file7 = "data/covid-confirmed-cases-since-100th-case.csv"

# Read File and store into Pandas data frame

density_vs_prosperity = pd.read_csv(file1)
urban_rural_population = pd.read_csv(file2)
urban_vs_rural_majority = pd.read_csv(file3)
life_expectancy = pd.read_csv(file4)
median_age = pd.read_csv(file5)
hoefstede = pd.read_csv(file6)
covid_data = pd.read_csv(file7)

In [2]:
density_vs_prosperity.shape

(47234, 6)

In [3]:
density_vs_prosperity.dropna(subset=['Code'],inplace=True)
density_vs_prosperity.shape

(44392, 6)

In [4]:
urban_rural_population.dropna(subset=['Code'],inplace=True)
urban_rural_population.shape

(12363, 5)

In [5]:
urban_vs_rural_majority.dropna(subset=['Code'],inplace=True)
urban_vs_rural_majority.shape

(24019, 5)

In [6]:
life_expectancy.dropna(subset=['Code'],inplace=True)
life_expectancy.shape

(11125, 4)

In [7]:
median_age.dropna(subset=['Code'],inplace=True)
median_age.shape

(6324, 4)

In [8]:
# Merge data
merged_data = pd.merge(density_vs_prosperity, urban_rural_population, on=["Entity","Code", "Year"])
merged_data.shape

(12283, 8)

In [9]:
merged_data = pd.merge(merged_data, urban_vs_rural_majority, on=["Entity", "Code", "Year"])
merged_data.shape

(12283, 10)

In [10]:
merged_data = pd.merge(merged_data, life_expectancy, on=["Entity", "Code", "Year"])
merged_data.shape

(11007, 11)

In [11]:
merged_data.head()

,Entity,Code,Year,Population density (people per km² of land area),GDP per capita (constant 2011 international $),Total population (Gapminder),Urban population,Rural population,Urban (%),Rural (%),"Life expectancy at birth, total (years) (years)"
0,Afghanistan,AFG,1960,NaN,NaN,8996351.0,755783,8240568,8.401,91.598999,32.292
1,Afghanistan,AFG,1961,14.040934,NaN,9166764.0,796042,8370722,8.684,91.316002,32.742
2,Afghanistan,AFG,1962,14.315271,NaN,9345868.0,838885,8506983,8.976,91.024002,33.185
3,Afghanistan,AFG,1963,14.603367,NaN,9533954.0,884370,8649584,9.276,90.723999,33.624
4,Afghanistan,AFG,1964,14.905739,NaN,9731361.0,932848,8798513,9.586,90.414001,34.060


In [12]:
merged_data = pd.merge(merged_data, median_age, on=["Entity", "Code", "Year"])
merged_data.shape

(2280, 12)

In [13]:
#Format columns
merged_data = merged_data.rename(columns={"Population density (people per km² of land area)": "pop_density",
                                          "GDP per capita (constant 2011 international $)" : "gdp",
                                         "Total population (Gapminder)": "total_pop",
                                         "Urban population": "urban_pop",
                                         "Rural population": "rural_pop",
                                         "Urban (%)":"urban_pct",
                                         "Rural (%)": "rural_pct",
                                         "Life expectancy at birth, total (years) (years)":"life_expectancy",
                                         "UN Population Division (Median Age) (2017) (years)":"median_age"})
merged_data.head()

,Entity,Code,Year,pop_density,gdp,total_pop,urban_pop,rural_pop,urban_pct,rural_pct,life_expectancy,median_age
0,Afghanistan,AFG,1960,NaN,NaN,8996351.0,755783,8240568,8.401,91.598999,32.292,18.799999
1,Afghanistan,AFG,1965,15.222887,NaN,9938414.0,984301,8954113,9.904,90.096001,34.495,18.400000
2,Afghanistan,AFG,1970,17.042127,NaN,11126123.0,1295415,9830708,11.643,88.357002,36.678,17.900000
3,Afghanistan,AFG,1975,19.284818,NaN,12590286.0,1717441,10872845,13.641,86.359001,39.039,17.400000
4,Afghanistan,AFG,1980,20.292819,NaN,13248370.0,2119077,11129293,15.995,84.004997,41.853,17.000000


In [14]:
hoefstede.shape

(127, 8)

In [15]:
hoefstede.head()

,ctr,country,pdi,idv,mas,uai,ltowvs,ivr
0,GTM,Guatemala,95,6,37,101,#NULL!,#NULL!
1,ECU,Ecuador,78,8,63,67,#NULL!,#NULL!
2,PAN,Panama,95,11,44,86,#NULL!,#NULL!
3,VEN,Venezuela,81,12,73,76,16,100
4,COL,Colombia,67,13,64,80,13,83


In [16]:
hoefstede = hoefstede.rename(columns={"ctr": "Code",
                                          "country" : "Entity"})
hoefstede.head()

,Code,Entity,pdi,idv,mas,uai,ltowvs,ivr
0,GTM,Guatemala,95,6,37,101,#NULL!,#NULL!
1,ECU,Ecuador,78,8,63,67,#NULL!,#NULL!
2,PAN,Panama,95,11,44,86,#NULL!,#NULL!
3,VEN,Venezuela,81,12,73,76,16,100
4,COL,Colombia,67,13,64,80,13,83


In [17]:
covid_data.shape

(12325, 5)

In [18]:
merged_data = pd.merge(merged_data, hoefstede, on=[ "Code"])
merged_data.head()

,Entity_x,Code,Year,pop_density,gdp,total_pop,urban_pop,rural_pop,urban_pct,rural_pct,life_expectancy,median_age,Entity_y,pdi,idv,mas,uai,ltowvs,ivr
0,Albania,ALB,1960,NaN,NaN,1636056.0,493982,1114818,30.705,69.294998,62.279,20.200001,Albania,90,20,80,70,61,15
1,Albania,ALB,1965,68.058066,NaN,1896125.0,582374,1282417,31.230,68.769997,65.848,19.700001,Albania,90,20,80,70,61,15
2,Albania,ALB,1970,77.937190,NaN,2150598.0,677801,1457678,31.740,68.260002,66.933,19.600000,Albania,90,20,80,70,61,15
3,Albania,ALB,1975,87.767555,NaN,2411229.0,786668,1618163,32.712,67.288002,68.734,20.200001,Albania,90,20,80,70,61,15
4,Albania,ALB,1980,97.518139,NaN,2681239.0,902120,1769877,33.762,66.237999,70.207,21.400000,Albania,90,20,80,70,61,15


In [19]:
merged_data.shape

(1381, 19)

In [20]:
merged_data['Year'].max()

2015

In [21]:
merged_data = merged_data[merged_data["Year"] == 2015]
merged_data.head()

,Entity_x,Code,Year,pop_density,gdp,total_pop,urban_pop,rural_pop,urban_pct,rural_pct,life_expectancy,median_age,Entity_y,pdi,idv,mas,uai,ltowvs,ivr
11,Albania,ALB,2015,105.135146,10970.452245,2923352.0,1654503,1226200,57.434,42.566002,78.174,36.200001,Albania,90,20,80,70,61,15
23,Algeria,DZA,2015,16.740504,13724.723853,39871528.0,28248180,11623348,70.848,29.152000,75.860,27.500000,Algeria,#NULL!,#NULL!,#NULL!,#NULL!,26,32
35,Angola,AGO,2015,22.346439,6227.664689,27859305.0,17675615,10183690,63.446,36.554001,61.241,16.400000,Angola,83,18,20,60,15,83
47,Argentina,ARG,2015,15.865065,19116.907915,43417765.0,39728558,3689207,91.503,8.497000,76.415,30.799999,Argentina,49,46,56,86,20,62
59,Armenia,ARM,2015,102.456972,8195.933650,2916950.0,1840158,1076792,63.085,36.915001,74.445,33.900002,Armenia,#NULL!,#NULL!,#NULL!,#NULL!,61,#NULL!


In [22]:
del merged_data['Entity_y']

In [23]:
merged_data = merged_data.rename(columns={"Entity_x": "Entity"})

In [24]:
merged_data.head()

,Entity,Code,Year,pop_density,gdp,total_pop,urban_pop,rural_pop,urban_pct,rural_pct,life_expectancy,median_age,pdi,idv,mas,uai,ltowvs,ivr
11,Albania,ALB,2015,105.135146,10970.452245,2923352.0,1654503,1226200,57.434,42.566002,78.174,36.200001,90,20,80,70,61,15
23,Algeria,DZA,2015,16.740504,13724.723853,39871528.0,28248180,11623348,70.848,29.152000,75.860,27.500000,#NULL!,#NULL!,#NULL!,#NULL!,26,32
35,Angola,AGO,2015,22.346439,6227.664689,27859305.0,17675615,10183690,63.446,36.554001,61.241,16.400000,83,18,20,60,15,83
47,Argentina,ARG,2015,15.865065,19116.907915,43417765.0,39728558,3689207,91.503,8.497000,76.415,30.799999,49,46,56,86,20,62
59,Armenia,ARM,2015,102.456972,8195.933650,2916950.0,1840158,1076792,63.085,36.915001,74.445,33.900002,#NULL!,#NULL!,#NULL!,#NULL!,61,#NULL!


In [25]:
merged_data.shape

(116, 18)

In [26]:
# Export CSV
merged_data.to_csv('merged_data_modified.csv')  

In [27]:
merged_data.tail()

,Entity,Code,Year,pop_density,gdp,total_pop,urban_pop,rural_pop,urban_pct,rural_pct,life_expectancy,median_age,pdi,idv,mas,uai,ltowvs,ivr
1332,Uruguay,URY,2015,19.606628,19831.445899,3431552.0,3261519,170033,95.045,4.955000,77.341,34.900002,61,36,38,100,26,53
1344,Venezuela,VEN,2015,35.321279,NaN,31155134.0,27464497,3690637,88.154,11.846000,74.367,27.400000,81,12,73,76,16,100
1356,Vietnam,VNM,2015,301.775622,5554.858056,93571567.0,31635611,61935956,33.809,66.191002,76.052,30.400000,70,20,40,30,57,35
1368,Zambia,ZMB,2015,21.658331,3627.202041,16100587.0,6747273,9353314,41.907,58.092999,61.397,17.100000,60,35,40,50,30,42
1380,Zimbabwe,ZWE,2015,40.784415,1912.280261,15777451.0,5109528,10667923,32.385,67.614998,60.398,19.000000,#NULL!,#NULL!,#NULL!,#NULL!,15,28
